In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import cartopy.crs as ccrs
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions

from pycontrails.physics import units
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)
from init_chem import CHEM
from chem import ChemDataset
from boxm import BoxModel

In [ ]:
# Initialise coord arrays
lon_bounds = (0, 5) #np.arange(-180, 180, 5)
lat_bounds = (0, 5) #np.arange(-90, 90, 5)
alt_bounds = (8000, 12000)
horiz_res = 2.5
vert_res = 500
met_pressure_levels = np.array([400, 300, 200, 100])

time = ("2022-01-01 00:00:00", "2022-01-01 08:00:00")
ts_met = "1H"
ts_disp = "1min"
ts_chem = "1H"

runtime = "24H"

lons = [0]
lats = [0]
alts = [11000]

# lons = np.arange(lon_bounds[0], lon_bounds[1], horiz_res)
# lats = np.arange(lat_bounds[0], lat_bounds[1], horiz_res)
# alts = np.arange(alt_bounds[0], alt_bounds[1], vert_res)

In [ ]:
# Import met data from ERA5
era5 = ERA5(
        time=time,
        timestep_freq=ts_met,
        variables=[
                "t",
                "q",
                "u",
                "v",
                "w",
                "z",
                "relative_humidity"
        ],
        pressure_levels=met_pressure_levels
)

In [ ]:
# download data from ERA5 (or open from cache)
met = era5.open_metdataset()
met.data = met.data.transpose("latitude", "longitude", "level", "time", ...)
met


In [ ]:
# Initialise low-res chem xarray dataset, and import species files
init_chem = CHEM(
        time=time,
        timestep_freq=ts_chem,
)

init_chem.timesteps

In [ ]:
# Populate chem dataset with species data
chem = init_chem.open_chemdataset()

In [ ]:
met.data = met.data.interp(longitude=lons, latitude=lats, level=units.m_to_pl(alts), method="linear")

chem.data = chem.data.interp(longitude=lons, latitude=lats, level=units.m_to_pl(alts), method="linear")

In [ ]:
met

In [ ]:
chem.zenith()
chem.get_photol_params()

In [ ]:
# p = chem["sza"].data.sel(time=chem.data.time[0]).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(20, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# p.axes.set_global()
# p.axes.coastlines()


In [ ]:
# chem_plot = chem.data["Y"].sel(species="NO", time=time[0], level=chem.data.level[0]).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(-50, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# chem_plot.axes.set_global()
# chem_plot.axes.coastlines()

In [ ]:
# import cartopy.crs as ccrs

# p = met.data["air_temperature"].isel(level=0, time=0).plot(
#     subplot_kws=dict(projection=ccrs.Orthographic(20, 10), facecolor="gray"),
#     transform=ccrs.PlateCarree(),
# )

# p.axes.set_global()

# p.axes.coastlines()

# # save figure
# p.figure.savefig(str(lon_bounds) + str(lat_bounds) + ".png")

# type(p)

In [ ]:
#chem.data["Y"].sel(species="DUMMY2", time=time[0], level=962).plot()

In [ ]:
boxm = BoxModel(met, chem)

In [ ]:
boxm.met

In [ ]:
boxm.chem

In [ ]:
emi = xr.Dataset(
    {
        "EM": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(lats), len(lons), len(alts), len(chem.data.time), len(chem.data.species)))),
    },
    coords={
        "latitude": lats,
        "longitude": lons, 
        "level": alts,
        "time": chem.data.time,
        "species": chem.data.species,
    }
)

In [ ]:
emi = ChemDataset(emi)
emi.data = emi.data.transpose("latitude", "longitude", "level", "time", ...)

In [ ]:
emi

In [ ]:
boxm.params

In [ ]:
boxm.eval(source=emi)

In [ ]:
boxm.chem.data["Y"].sel(species="OH").shape